In [ ]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import lpips
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from diffusers import AutoencoderKL
from transformers import CLIPFeatureExtractor
from torchvision.transforms import functional as TF

In [ ]:
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")
lpips_model = lpips.LPIPS(net='alex')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 110MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(model_path, map_location='cpu'), st

In [ ]:
def load_image(path):
    image = Image.open(path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform(image).unsqueeze(0)

In [ ]:
def reconstruct_image(image_path):
    input_image = Image.open(image_path).convert("RGB")
    inputs = feature_extractor(images=input_image, return_tensors="pt")
    input_tensor = inputs.pixel_values
    with torch.no_grad():
        latent = vae.encode(input_tensor).latent_dist.sample()
        reconstructed_image = vae.decode(latent).sample
    # Resize reconstructed image to match input size (256x256)
    reconstructed_image_resized = TF.resize(reconstructed_image.squeeze(0), (256, 256))
    return reconstructed_image_resized

In [ ]:
def calculate_lpips(image_path):
    try:
        original_image = load_image(image_path)
        reconstructed_image = reconstruct_image(image_path)
        # Ensure tensors have matching dimensions
        reconstructed_image = reconstructed_image.unsqueeze(0)  # Add batch dimension
        lpips_distance = lpips_model(original_image, reconstructed_image)
        return lpips_distance.item()
    except Exception as e:
        print(f"Error calculating LPIPS for {image_path}: {e}")
        return None

In [ ]:
def process_dataset(base_path, output_csv, max_samples_per_class=1000):
    data = []
    class_sample_counts = {0: 0, 1: 0}  # Track samples for each class (real: 0, fake: 1)

    for class_label, class_folder in enumerate(["REAL_temp", "FAKE_temp"]):
        folder_path = os.path.join(base_path, class_folder)
        for filename in os.listdir(folder_path):
            if class_sample_counts[class_label] >= max_samples_per_class:
                break  # Stop processing this class if the limit is reached
            if filename.endswith(('.jpg', '.png', '.jpeg')):
                image_path = os.path.join(folder_path, filename)
                lpips_distance = calculate_lpips(image_path)
                if lpips_distance is not None:
                    data.append({"Image": filename, "LPIPS_Distance": lpips_distance, "Class": class_label})
                    class_sample_counts[class_label] += 1

    # Save to CSV
    df = pd.DataFrame(data)
    df.to_csv(output_csv, index=False)
    return df

In [ ]:
def perform_logistic_regression(csv_file):
    # Load data
    df = pd.read_csv(csv_file)
    X = df["LPIPS_Distance"].values.reshape(-1, 1)
    y = df["Class"].values

    # Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train logistic regression
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))


In [ ]:

    base_path = '/content/drive/MyDrive/adobe_interiit/train'  # Update with your dataset path
    output_csv = 'lpips_results.csv'
    max_samples_per_class = 1000
    # Process dataset and generate CSV
    print("Processing dataset...")
    df = process_dataset(base_path, output_csv, max_samples_per_class)
    print(f"CSV saved at {output_csv}")

    # Perform logistic regression
    print("Performing Logistic Regression...")
    perform_logistic_regression(output_csv)

Processing dataset...
